In [42]:
import os
import json
import datetime
import time

from dateutil import parser
import glob
import json
from dataclasses import dataclass, field
from dacite import from_dict, Config
from typing import Optional

CREDENTIALS_FILE = "../creds.txt"

EXP = "tokyo"
USERFILE = f"../datasets/users/ia/{EXP}.json"


OUTDIR = f"../datasets/queried/{EXP}"
QUERY_RESULTS = "users.jsonl"
TWEET_QUERY_RESULTS = "tweets.jsonl"
MAX_USERS = 5000
# CONTROL = True

# if CONTROL:
#     USERDIR = f"{USERDIR}-control"
#     QUERY_RESULTS = f"control_{QUERY_RESULTS}"
#     TWEET_QUERY_RESULTS = f"control_{TWEET_QUERY_RESULTS}"

QUERY_RESULTS = os.path.join(OUTDIR, QUERY_RESULTS)
TWEET_QUERY_RESULTS = os.path.join(OUTDIR, TWEET_QUERY_RESULTS)

In [43]:
@dataclass
class Tweet:
    id: int
    text: str
    created_at: str
    lang: str
    source: str
    retweeted: bool

@dataclass
class User:
    id: int
    screen_name: str
    name: str
    description: Optional[str]
    location: Optional[str]
    tweets: list[Tweet] = field(default_factory=list)

In [44]:
# Load all files in USERDIR and read user objects
users = {}
tweets = set([])
with open(USERFILE, "r") as f:
    for line in f:
        userdata = from_dict(data_class=User, data=json.loads(line.strip()))
        if userdata.id not in users:
            users[userdata.id] = userdata
        else:
            users[userdata.id].tweets += userdata.tweets
        tweets.update([tweet.id for tweet in userdata.tweets])

In [45]:
len(users)
QUERY_RESULTS

'../datasets/queried/tokyo/users.jsonl'

# Twitter API

In [46]:
# Load Twitter API OAuth and other details

from TwitterAPI import TwitterAPI, TwitterOAuth

o = TwitterOAuth.read_file(CREDENTIALS_FILE)
api = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret, api_version="2")


def batch(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

## User Queries

Here, we batch-query Tweet IDs to see if they still exist. We skip Tweet IDs for which we know the Tweeter either has a protected or non-existent account.

In [47]:
# Make sure we don't repeat queries to IDs we have already queried Twitter's API for!
if os.path.exists(QUERY_RESULTS):
    with open(QUERY_RESULTS, "r") as f:
        for line in f:
            line = line.strip()
            id = json.loads(line)["id"]
            if id in users:
                del users[id]

In [48]:
print(len(users))

215219


In [49]:
# Send user queries to Twitter in batches of 100.
userids = list(users.keys())
user_responses = {}

for id_batch in batch(userids, 100):
    ids = ",".join([str(id) for id in id_batch])
    params = {
        "ids": ids,
        "user.fields": "id,description,location,name,protected,verified,withheld,username"
    }
    r = api.request(f"users", params)
    for item in r:
        user_responses[int(item["id"])] = item
    for id in id_batch:
        if id not in user_responses:
            user_responses[id] = "Not found"
    if r.get_quota()["remaining"] < 1:
        print("Ran into quota, sleeping for 15 mins")
        time.sleep(15*60)
    print(r.get_quota())


{'remaining': 899, 'limit': None, 'reset': None}
{'remaining': 898, 'limit': None, 'reset': None}
{'remaining': 897, 'limit': None, 'reset': None}
{'remaining': 896, 'limit': None, 'reset': None}
{'remaining': 895, 'limit': None, 'reset': None}
{'remaining': 894, 'limit': None, 'reset': None}
{'remaining': 893, 'limit': None, 'reset': None}
{'remaining': 892, 'limit': None, 'reset': None}
{'remaining': 891, 'limit': None, 'reset': None}
{'remaining': 890, 'limit': None, 'reset': None}
{'remaining': 889, 'limit': None, 'reset': None}
{'remaining': 888, 'limit': None, 'reset': None}
{'remaining': 887, 'limit': None, 'reset': None}
{'remaining': 886, 'limit': None, 'reset': None}
{'remaining': 885, 'limit': None, 'reset': None}
{'remaining': 884, 'limit': None, 'reset': None}
{'remaining': 883, 'limit': None, 'reset': None}
{'remaining': 882, 'limit': None, 'reset': None}
{'remaining': 881, 'limit': None, 'reset': None}
{'remaining': 880, 'limit': None, 'reset': None}
{'remaining': 879, '

{'remaining': 731, 'limit': None, 'reset': None}
{'remaining': 730, 'limit': None, 'reset': None}
{'remaining': 729, 'limit': None, 'reset': None}
{'remaining': 728, 'limit': None, 'reset': None}
{'remaining': 727, 'limit': None, 'reset': None}
{'remaining': 726, 'limit': None, 'reset': None}
{'remaining': 725, 'limit': None, 'reset': None}
{'remaining': 724, 'limit': None, 'reset': None}
{'remaining': 723, 'limit': None, 'reset': None}
{'remaining': 722, 'limit': None, 'reset': None}
{'remaining': 721, 'limit': None, 'reset': None}
{'remaining': 720, 'limit': None, 'reset': None}
{'remaining': 719, 'limit': None, 'reset': None}
{'remaining': 718, 'limit': None, 'reset': None}
{'remaining': 717, 'limit': None, 'reset': None}
{'remaining': 716, 'limit': None, 'reset': None}
{'remaining': 715, 'limit': None, 'reset': None}
{'remaining': 714, 'limit': None, 'reset': None}
{'remaining': 713, 'limit': None, 'reset': None}
{'remaining': 712, 'limit': None, 'reset': None}
{'remaining': 711, '

{'remaining': 563, 'limit': None, 'reset': None}
{'remaining': 562, 'limit': None, 'reset': None}
{'remaining': 561, 'limit': None, 'reset': None}
{'remaining': 560, 'limit': None, 'reset': None}
{'remaining': 559, 'limit': None, 'reset': None}
{'remaining': 558, 'limit': None, 'reset': None}
{'remaining': 557, 'limit': None, 'reset': None}
{'remaining': 556, 'limit': None, 'reset': None}
{'remaining': 555, 'limit': None, 'reset': None}
{'remaining': 554, 'limit': None, 'reset': None}
{'remaining': 553, 'limit': None, 'reset': None}
{'remaining': 552, 'limit': None, 'reset': None}
{'remaining': 551, 'limit': None, 'reset': None}
{'remaining': 550, 'limit': None, 'reset': None}
{'remaining': 549, 'limit': None, 'reset': None}
{'remaining': 548, 'limit': None, 'reset': None}
{'remaining': 547, 'limit': None, 'reset': None}
{'remaining': 546, 'limit': None, 'reset': None}
{'remaining': 545, 'limit': None, 'reset': None}
{'remaining': 544, 'limit': None, 'reset': None}
{'remaining': 543, '

{'remaining': 395, 'limit': None, 'reset': None}
{'remaining': 394, 'limit': None, 'reset': None}
{'remaining': 393, 'limit': None, 'reset': None}
{'remaining': 392, 'limit': None, 'reset': None}
{'remaining': 391, 'limit': None, 'reset': None}
{'remaining': 390, 'limit': None, 'reset': None}
{'remaining': 389, 'limit': None, 'reset': None}
{'remaining': 388, 'limit': None, 'reset': None}
{'remaining': 387, 'limit': None, 'reset': None}
{'remaining': 386, 'limit': None, 'reset': None}
{'remaining': 385, 'limit': None, 'reset': None}
{'remaining': 384, 'limit': None, 'reset': None}
{'remaining': 383, 'limit': None, 'reset': None}
{'remaining': 382, 'limit': None, 'reset': None}
{'remaining': 381, 'limit': None, 'reset': None}
{'remaining': 380, 'limit': None, 'reset': None}
{'remaining': 379, 'limit': None, 'reset': None}
{'remaining': 378, 'limit': None, 'reset': None}
{'remaining': 377, 'limit': None, 'reset': None}
{'remaining': 376, 'limit': None, 'reset': None}
{'remaining': 375, '

{'remaining': 227, 'limit': None, 'reset': None}
{'remaining': 226, 'limit': None, 'reset': None}
{'remaining': 225, 'limit': None, 'reset': None}
{'remaining': 224, 'limit': None, 'reset': None}
{'remaining': 223, 'limit': None, 'reset': None}
{'remaining': 222, 'limit': None, 'reset': None}
{'remaining': 221, 'limit': None, 'reset': None}
{'remaining': 220, 'limit': None, 'reset': None}
{'remaining': 219, 'limit': None, 'reset': None}
{'remaining': 218, 'limit': None, 'reset': None}
{'remaining': 217, 'limit': None, 'reset': None}
{'remaining': 216, 'limit': None, 'reset': None}
{'remaining': 215, 'limit': None, 'reset': None}
{'remaining': 214, 'limit': None, 'reset': None}
{'remaining': 213, 'limit': None, 'reset': None}
{'remaining': 212, 'limit': None, 'reset': None}
{'remaining': 211, 'limit': None, 'reset': None}
{'remaining': 210, 'limit': None, 'reset': None}
{'remaining': 209, 'limit': None, 'reset': None}
{'remaining': 208, 'limit': None, 'reset': None}
{'remaining': 207, '

...

{'remaining': 454, 'limit': None, 'reset': None}
{'remaining': 453, 'limit': None, 'reset': None}
{'remaining': 452, 'limit': None, 'reset': None}
{'remaining': 451, 'limit': None, 'reset': None}
{'remaining': 450, 'limit': None, 'reset': None}
{'remaining': 449, 'limit': None, 'reset': None}
{'remaining': 448, 'limit': None, 'reset': None}
{'remaining': 447, 'limit': None, 'reset': None}
{'remaining': 446, 'limit': None, 'reset': None}
{'remaining': 445, 'limit': None, 'reset': None}
{'remaining': 444, 'limit': None, 'reset': None}
{'remaining': 443, 'limit': None, 'reset': None}
{'remaining': 442, 'limit': None, 'reset': None}
{'remaining': 441, 'limit': None, 'reset': None}
{'remaining': 440, 'limit': None, 'reset': None}
{'remaining': 439, 'limit': None, 'reset': None}
{'remaining': 438, 'limit': None, 'reset': None}
{'remaining': 437, 'limit': None, 'reset': None}
{'remaining': 436, 'limit': None, 'reset': None}
{'remaining': 435, 'limit': None, 'reset': None}
{'remaining': 434, '

{'remaining': 286, 'limit': None, 'reset': None}
{'remaining': 285, 'limit': None, 'reset': None}
{'remaining': 284, 'limit': None, 'reset': None}
{'remaining': 283, 'limit': None, 'reset': None}
{'remaining': 282, 'limit': None, 'reset': None}
{'remaining': 281, 'limit': None, 'reset': None}
{'remaining': 280, 'limit': None, 'reset': None}
{'remaining': 279, 'limit': None, 'reset': None}
{'remaining': 278, 'limit': None, 'reset': None}
{'remaining': 277, 'limit': None, 'reset': None}
{'remaining': 276, 'limit': None, 'reset': None}
{'remaining': 275, 'limit': None, 'reset': None}
{'remaining': 274, 'limit': None, 'reset': None}
{'remaining': 273, 'limit': None, 'reset': None}
{'remaining': 272, 'limit': None, 'reset': None}
{'remaining': 271, 'limit': None, 'reset': None}
{'remaining': 270, 'limit': None, 'reset': None}
{'remaining': 269, 'limit': None, 'reset': None}
{'remaining': 268, 'limit': None, 'reset': None}
{'remaining': 267, 'limit': None, 'reset': None}
{'remaining': 266, '

{'remaining': 118, 'limit': None, 'reset': None}
{'remaining': 117, 'limit': None, 'reset': None}
{'remaining': 116, 'limit': None, 'reset': None}
{'remaining': 115, 'limit': None, 'reset': None}
{'remaining': 114, 'limit': None, 'reset': None}
{'remaining': 113, 'limit': None, 'reset': None}
{'remaining': 112, 'limit': None, 'reset': None}
{'remaining': 111, 'limit': None, 'reset': None}
{'remaining': 110, 'limit': None, 'reset': None}
{'remaining': 109, 'limit': None, 'reset': None}
{'remaining': 108, 'limit': None, 'reset': None}
{'remaining': 107, 'limit': None, 'reset': None}
{'remaining': 106, 'limit': None, 'reset': None}
{'remaining': 105, 'limit': None, 'reset': None}
{'remaining': 104, 'limit': None, 'reset': None}
{'remaining': 103, 'limit': None, 'reset': None}
{'remaining': 102, 'limit': None, 'reset': None}
{'remaining': 101, 'limit': None, 'reset': None}
{'remaining': 100, 'limit': None, 'reset': None}
{'remaining': 99, 'limit': None, 'reset': None}
{'remaining': 98, 'li

{'remaining': 850, 'limit': None, 'reset': None}
{'remaining': 849, 'limit': None, 'reset': None}
{'remaining': 848, 'limit': None, 'reset': None}
{'remaining': 847, 'limit': None, 'reset': None}
{'remaining': 846, 'limit': None, 'reset': None}
{'remaining': 845, 'limit': None, 'reset': None}
{'remaining': 844, 'limit': None, 'reset': None}
{'remaining': 843, 'limit': None, 'reset': None}
{'remaining': 842, 'limit': None, 'reset': None}
{'remaining': 841, 'limit': None, 'reset': None}
{'remaining': 840, 'limit': None, 'reset': None}
{'remaining': 839, 'limit': None, 'reset': None}
{'remaining': 838, 'limit': None, 'reset': None}
{'remaining': 837, 'limit': None, 'reset': None}
{'remaining': 836, 'limit': None, 'reset': None}
{'remaining': 835, 'limit': None, 'reset': None}
{'remaining': 834, 'limit': None, 'reset': None}
{'remaining': 833, 'limit': None, 'reset': None}
{'remaining': 832, 'limit': None, 'reset': None}
{'remaining': 831, 'limit': None, 'reset': None}
{'remaining': 830, '

{'remaining': 682, 'limit': None, 'reset': None}
{'remaining': 681, 'limit': None, 'reset': None}
{'remaining': 680, 'limit': None, 'reset': None}
{'remaining': 679, 'limit': None, 'reset': None}
{'remaining': 678, 'limit': None, 'reset': None}
{'remaining': 677, 'limit': None, 'reset': None}
{'remaining': 676, 'limit': None, 'reset': None}
{'remaining': 675, 'limit': None, 'reset': None}
{'remaining': 674, 'limit': None, 'reset': None}
{'remaining': 673, 'limit': None, 'reset': None}
{'remaining': 672, 'limit': None, 'reset': None}
{'remaining': 671, 'limit': None, 'reset': None}
{'remaining': 670, 'limit': None, 'reset': None}
{'remaining': 669, 'limit': None, 'reset': None}
{'remaining': 668, 'limit': None, 'reset': None}
{'remaining': 667, 'limit': None, 'reset': None}
{'remaining': 666, 'limit': None, 'reset': None}
{'remaining': 665, 'limit': None, 'reset': None}
{'remaining': 664, 'limit': None, 'reset': None}
{'remaining': 663, 'limit': None, 'reset': None}
{'remaining': 662, '

In [50]:
len(user_responses)

215219

In [51]:
# Append any new results to QUERY_RESULTS.

with open(QUERY_RESULTS, "a") as f:
    for user, response in user_responses.items():
        if response == "Not found":
            response = {"found": False}
        else:
            response["found"] = True
        response["id"] = user
        response["queried_time"] = str(datetime.datetime.now())
        json.dump(response, f)
        f.write("\n")

## Tweet Queries

Here, we batch-query Tweet IDs to see if they still exist. We skip Tweet IDs for which we know the Tweeter either has a protected or non-existent account.

In [56]:
# Make sure we don't repeat queries to IDs we have already queried Twitter's API for!
if os.path.exists(TWEET_QUERY_RESULTS):
    with open(TWEET_QUERY_RESULTS, "r") as f:
        for line in f:
            line = line.strip()
            id = json.loads(line)["id"]
            if id in tweets:
                tweets.remove(id)
# Don't query tweets w/ deleted or protected users
if os.path.exists(QUERY_RESULTS):
    with open(QUERY_RESULTS, "r") as f:
        for line in f:
            datum = json.loads(line.strip())
            id = datum["id"]
            if id not in users:
                continue
            if not datum["found"] or datum["protected"]:
                for tweet in users[id].tweets:
                    if tweet.id in tweets:
                        tweets.remove(tweet.id)
print(len(tweets))

284481


In [ ]:
tweet_responses = {}
for batch_tweets in batch(list(tweets), 100):
    ids = ",".join([str(twt_id) for twt_id in batch_tweets])
    params = {
        "ids": ids,
        "tweet.fields": "id,author_id,withheld"
    }
    r = api.request(f"tweets", params)
    for item in r:
        item["queried_time"] = str(datetime.datetime.now())
        item["found"] = True
        tweet_responses[int(item["id"])] = item
    for twt_id in batch_tweets:
        if twt_id not in tweet_responses:
            tweet_responses[twt_id] = {"found": False, "queried_time": str(datetime.datetime.now())}
    if r.get_quota()["remaining"] < 1:
        print("Ran into quota, sleeping for 15 mins")
        time.sleep(15*60)
    print(r.get_quota()["remaining"])


899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750
749
748
747
746
745
744
743
742
741
740
739
738
737
736
735
734
733
732
731
730
729
728
727
726
725
724
723
722
721
720
719
718
717
716
715
714
713
712
711
710
709
708
707
706
705
704
703
702
701
700
699
698
697
696
695
694
693
692
691
690
689
688
687
686
685
684
683
682
681
680
679
678
677
676
675
674
673
672
671
670
669
668
667
666
665
664
663
662
661
660
659
658
657
656
655
654
653
652
651
650


In [55]:
with open(TWEET_QUERY_RESULTS, "a") as f:
    for tweet, response in tweet_responses.items():
        response["id"] = tweet
        json.dump(response, f)
        f.write("\n")
TWEET_QUERY_RESULTS

'../datasets/queried/tokyo/tweets.jsonl'

In [54]:
len(tweet_responses)

270000